In [7]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import requests
import urllib.request

In [8]:
# 정제형 data
main_url = 'https://nedrug.mfds.go.kr/searchDrug'

driver = webdriver.Chrome(executable_path="/Users/jangdong-geun/Study/BigData_myoungji/chromedriver")
driver.get(main_url)
driver.find_element_by_css_selector('#con_body > div.mediWrap.m-search > div.l_sec_md > div.ltab > ul > li:nth-child(2) > a').click()
driver.find_element_by_css_selector('#shapeCodeList > li.l02 > a').click()
driver.find_element_by_css_selector('#searchGrain > fieldset > div.search_part.btn_part > button.btn_search100').click()


In [9]:
# 정제형 총 개수
totalCount = driver.find_element_by_css_selector('#con_body > div.mediWrap.m-search > div.r_sec_md > div.search_top.mb10 > p > strong')
print('검색 결과:', totalCount.text)
totalCount = int(totalCount.text.replace(',', ''))
totalCount

검색 결과: 20,975


20975

In [4]:
# 페이지 개수
totalPage = (totalCount // 15) + 1
totalPage

1399

In [10]:
pill_name = []
# pill_img = []
pill_link = []
pill_code = []

In [11]:
for page in range(300, 350):
    try:
        # 페이지 이동
        driver.execute_script('javascript:getList2(%s)' % page)
        time.sleep(2)
        print('%s 페이지 이동' %page)
        
        # 각 알약 별로 주소
        web_url = driver.page_source
        soup = BeautifulSoup(web_url, 'html.parser')
        trs = soup.find_all('tr')
        for tr in trs[1:]:
            pills = tr.find('td',{'class' : 'al_l'}).find('a')['href']
            
            try:
                r = requests.get('https://nedrug.mfds.go.kr%s' %pills)
                c = r.content
                soup_pill = BeautifulSoup(c, 'html.parser')
                
                # img
                ul = soup_pill.find('ul', {'class':'bxslider'})
                li = ul.find('li')
                img=li.find("img")
                
                # name
                div = soup_pill.find('div', {'class' : 'title'})
                h1 = div.find('h1')
                
                # code
                div_code = soup_pill.find('div', {'class' : 'r_sec'})
                tr = div_code.find_all('tr')
                for i in range(len(tr)):
                    if tr[i].find('th').text == '표준코드':
                        code = tr[i].find('td').text.split(',',1)[0]
                        
                pill_link.append(pills)
                pill_name.append(h1.text.replace('\n','').split('(',1)[0])
                # pill_img.append(img["src"])
                pill_code.append(code[3:-1])
                urllib.request.urlretrieve(img["src"], 'img3/'+ str(code[3:-1])+'.jpg')
                
                    
            except Exception as e1:
                print('오류', e1)
                
    except Exception as e1:
        print('오류', e1)

300 페이지 이동
오류 'NoneType' object has no attribute 'find'
오류 'NoneType' object has no attribute 'find'
오류 'NoneType' object has no attribute 'find'
301 페이지 이동
오류 'NoneType' object has no attribute 'find'
오류 'NoneType' object has no attribute 'find'
오류 'NoneType' object has no attribute 'find'
오류 'NoneType' object has no attribute 'find'
오류 'NoneType' object has no attribute 'find'
302 페이지 이동
오류 'NoneType' object has no attribute 'find'
오류 'NoneType' object has no attribute 'find'
오류 'NoneType' object has no attribute 'find'
오류 'NoneType' object has no attribute 'find'
오류 'NoneType' object has no attribute 'find'
303 페이지 이동
오류 'NoneType' object has no attribute 'find'
오류 'NoneType' object has no attribute 'find'
오류 'NoneType' object has no attribute 'find'
오류 'NoneType' object has no attribute 'find'
오류 'NoneType' object has no attribute 'find'
304 페이지 이동
오류 'NoneType' object has no attribute 'find'
305 페이지 이동
오류 'NoneType' object has no attribute 'find'
오류 'NoneType' object has no attrib

In [12]:
data = {'pill_name' : pill_name, 'pill_code' : pill_code, 'pill_link' : pill_link}

In [13]:
pill_Data = pd.DataFrame(data)

In [14]:
len(pill_Data)

560

In [16]:
import os
len(os.listdir('img3'))

557

In [21]:
pill_Data

,pill_code,pill_link,pill_name
0,640902430,/pbp/CCBBB01/getItemDetail?itemSeq=201310869,리론트정35밀리그램
1,640903270,/pbp/CCBBB01/getItemDetail?itemSeq=201604705,리론트플러스정
2,652000220,/pbp/CCBBB01/getItemDetail?itemSeq=200805962,리루텍정
3,642800470,/pbp/CCBBB01/getItemDetail?itemSeq=200308619,리리스정
4,648903390,/pbp/CCBBB01/getItemDetail?itemSeq=201802990,리리카CR서방정165밀리그램
5,648903400,/pbp/CCBBB01/getItemDetail?itemSeq=201802991,리리카CR서방정330밀리그램
6,653103340,/pbp/CCBBB01/getItemDetail?itemSeq=201800229,리릭스알정480밀리그램
7,656004180,/pbp/CCBBB01/getItemDetail?itemSeq=201602341,리마덱스정
8,643902750,/pbp/CCBBB01/getItemDetail?itemSeq=200708144,리마딘정
9,648506750,/pbp/CCBBB01/getItemDetail?itemSeq=201206497,리마로스정


In [17]:
(pill_Data[pill_Data['pill_code']==''])

,pill_code,pill_link,pill_name
303,,/pbp/CCBBB01/getItemDetail?itemSeq=201901774,리피테롤정20mg
375,,/pbp/CCBBB01/getItemDetail?itemSeq=201901764,마모빅사정
385,,/pbp/CCBBB01/getItemDetail?itemSeq=201903474,마사포서방정300밀리그램
420,,/pbp/CCBBB01/getItemDetail?itemSeq=201902614,마인트롤정


In [34]:
len(os.listdir('img3'))

557

In [20]:
pill_Data.to_csv('pill_Data(300-350).csv', index=False)

In [32]:
for i in range(len(os.listdir('img3'))):
    if 'DS' in os.listdir('img3')[i]:
        print(os.listdir('img3')[i])
print('X')

X
